The model we are hoping to build looks like: 

V_ij = eps_ij $*$ (V_ii $*$ e^(-i $*$ pi $*$ tau_ci $*$ nu) + V_jj $*$ e^-i $*$ pi $*$  tau_cj $*$ nu)) 
        + P_ij * e^(-i * pi * tau_aij * nu) * (V_ii + V_jj)

The variables we are working with are: 

V_ij : the data averaged across time

eps_ij : the amplitude of the signal traveling through cables

        (getting the approx from: the amplitude array from outside baseline dependence)

V_ii, V_jj : the autocorrelation for each of the two antennas

P_ij : the amplitude of the signal traveling through the air

        (getting the approx from: the amplitude array within baseline dependence)

tau_ci, tau_cj : the "cable" delays 

        (getting the approx from: a row or column of amplitude array averaged across frequency)

tau_aij : the air delay 
        
        (getting the approx from: baseline length in ns)

nu : the frequency

        (getting the approx from: the frequency array)

In [37]:
%matplotlib notebook
#Import needed packages
from pyuvdata import UVData
import numpy as np
import glob
import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
import cmath
#Import the functions I defined for this notebook
import amp_delay_plotting as adp

#Create uv elements
uvxy = UVData()
uvyx = UVData()
uvxx = UVData()
uvyy = UVData()

#Create constant variables
#Variable for the speed of light, with power adjusted for desired units
c = 0.299792
#Variable for speed of light in a cable correction factor, based on Google search
f = .83

#Create an array of antennas we want to flag
flagged_antennas = np.array([0,2,26,50,98,136])

In [38]:
#Read each of the data files into the uv objects

# Read in the xy data file
uvxy.read_uvfits('/data6/HERA/data/IDR2.1/uvOCRSL_crosspol_time_split/xy_time_split_data/combined_files/zen.grp1.of1.xy.LST.run_7.uvOCRSL.uvfits')

# Read in the yx data file
uvyx.read_uvfits('/data6/HERA/data/IDR2.1/uvOCRSL_crosspol_time_split/yx_time_split_data/combined_files/zen.grp1.of1.yx.LST.run_8.uvOCRSL.uvfits')

#Read in the xx data file
uvxx.read_uvfits('/data6/HERA/data/IDR2.1/uvOCRSL_crosspol_time_split/xx_time_split_data/combined_files/zen.grp1.of1.xx.LST.run_2.uvOCRSL.uvfits')

#Read in the yy data file
uvyy.read_uvfits('/data6/HERA/data/IDR2.1/uvOCRSL_yy_time_split_data/combined_files/zen.grp1.of1.yy.LST.run_2.uvOCRSL.uvfits')

In [39]:
#Read in the position and antenna numbers for all antennas
#These will be passed into functions to plot the antennas in the correct positions
antpos, ants = uvxy.get_ENU_antpos()

In [40]:
def find_bl_len(antnum1,antnum2):
    blin_length = 0
    #Step through each antenna pair with the entered antenna
    for aa,ant1 in enumerate(ants):
        for bb,ant2 in enumerate(ants):
            if (ant1 == antnum1 or ant1 == antnum2) and (ant2==antnum1 or ant2==antnum2):
                #Find the horizontal distance between the two antennas
                x = antpos[aa,0] - antpos[bb,0]
                #Find the vertical distance between the two antennas
                y = antpos[aa,1] - antpos[bb,1]
                #Find the hypotenus of the triangle, which is the total distance between
                #the two antennas
                diag = np.sqrt(x**2 + y**2)
                #Place this value into the array
                if diag != 0:
                    blin_length = diag
    return(blin_length)

In [51]:

#P_ij, blin_delay_xy = adp.make_blin_depd_arrays(uv = uvxy)
#eps_ij, ind_delay_xy = adp.make_blin_ind_arrays(uv = uvxy)

#Print the shape of the arrays to check that everything is working correctly
print blin_delay_xy.shape
print P_ij.shape
print eps_ij.shape
print ind_delay_xy.shape

#Find the length of the baseline
tau_aij = find_bl_len(11,12)
print tau_aij

#Convert meters to ns
tau_aij = tau_aij / c
print tau_aij

#Get the autocorr data for the antennas
V_ii = uvxy.get_data(11,11)
print V_ii.shape
V_jj = uvxy.get_data(12,12)
print V_jj.shape

#Get the data
V_ij = uvxy.get_data(11,12)
print V_ij.shape

nu = uvxy.freq_array
print nu.shape

(1035, 3)
(1035, 3)
(1035, 3)
(1035, 3)
14.607949891367397
48.72695032344892
(1, 1024)
(1, 1024)
(1, 1024)
(1, 1024)


In [50]:
#V_ij - (P_ij * np.e**(-1j * np.pi * tau_aij) * (V_ii + V_jj))
#Arrays aren't the same size and shape, so I can't actually do this yet
#Arrays need to be Nbls by Nfreqs, so can't use previously defined functions to get the data